In [ ]:
from numba import cuda
import numpy as np
from scipy.stats import norm
import torch
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam
import matplotlib.pyplot as plt
from time import time

device = torch.device('cuda:0')

In [ ]:
class net(nn.Module):
    def __init__(self, n_input, n_hidden1, n_hidden2, n_hidden3, n_output):
        """
        :param n_input: number of perceptrons for the input layer (int)
        :param n_hidden: number of perceptrons for the hidden layer (int)
        :param n_output: number of perceptrons for the output layer (int)
        """
        super().__init__()
        self.hidden1 = nn.Linear(n_input, n_hidden1)
        self.hidden2 = nn.Linear(n_hidden1, n_hidden2)
        self.hidden3 = nn.Linear(n_hidden2, n_hidden3)
        self.output = nn.Linear(n_hidden3, n_output)

    def forward(self, x):
        """
        :param x: Neural network input (2d torch tensor with arbitrary number of rows and n_input columns)
        :return: 2d torch tensor: Neural network output (same number of rows as x, n_output columns)
        """
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = F.relu(self.hidden3(x))
        x = self.output(x)
        return x  


In [ ]:
class net(nn.Module):
    def __init__(self, n_input, n_hidden1, n_hidden2, n_output):
        """
        :param n_input: number of perceptrons for the input layer (int)
        :param n_hidden: number of perceptrons for the hidden layer (int)
        :param n_output: number of perceptrons for the output layer (int)
        """
        super().__init__()
        self.hidden1 = nn.Linear(n_input, n_hidden1)
        self.hidden2 = nn.Linear(n_hidden1, n_hidden2)
        self.output = nn.Linear(n_hidden2, n_output)

    def forward(self, x):
        """
        :param x: Neural network input (2d torch tensor with arbitrary number of rows and n_input columns)
        :return: 2d torch tensor: Neural network output (same number of rows as x, n_output columns)
        """
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.output(x)
        return x  


In [ ]:
NC = norm.cdf #cumulative function of N(0,1), CPU version

def BS_price(S0, K, sig, r, T):
    """
    :param S0: time t stock price (stricly positive number)
    :param K: Call strike (stricly positive number)
    :param sig: volatility (stricly positive number)
    :param r: interest rate (stricly positive number)
    :param T: maturity time (stricly positive number)
    :return : BS price of call option at time 0 with maturity T by closed form formula
    """
    d1=(np.log(S0/K)+(r+sig**2/2)*T)/(sig*np.sqrt(T))
    d2=d1-sig*np.sqrt(T)
    return S0*NC(d1)-K*np.exp(-r*T)*NC(d2)

NC_GPU = torch.special.ndtr #cumulative function of N(0,1), GPU version

def BS_price_GPU(S0, K, sig, r, T):
    """
    :param S0: time t stock price (stricly positive number)
    :param K: Call strike (stricly positive number)
    :param sig: volatility (stricly positive number)
    :param r: interest rate (stricly positive number)
    :param T: maturity time (stricly positive number)
    :return : BS price of call option at time 0 with maturity T by closed form formula
    """
    d1=(torch.log(S0/K)+(r+sig**2/2)*T)/(sig*torch.sqrt(T))
    d2=d1-sig*torch.sqrt(T)
    return S0*NC_GPU(d1)-K*torch.exp(-r*T)*NC_GPU(d2)



In [ ]:
                                    """ CPU version"""

K = 100
T = 1
sig = 0.2
r = 0.1
batch_size = 2048
S0_min, S0_max = 50, 150

NET = net(1, 100, 100, 1)
#NET = net(1, 200, 200, 200, 1)
optimizer = Adam(NET.parameters(), lr=0.0005) 
loss_func = nn.MSELoss(reduction = 'mean')  

"""Uniform generation"""
t_ = time()
S0 = torch.tensor(np.random.uniform(S0_min, S0_max, batch_size*1024*2), dtype=torch.float)
print("(CPU) S0 generation time :", time()-t_)

S0 = S0.view(-1,1)
dataloader = torch.utils.data.DataLoader(S0, batch_size=batch_size, shuffle=True)

t_ = time()
for data in dataloader :
    optimizer.zero_grad()
    TRUE_price = BS_price(data, K, sig, r, T).float()
    NET_price = NET(data)
    loss = loss_func(TRUE_price, NET_price)
    #loss = torch.mean((TRUE_price - NET_price)**2)
    loss.backward()
    optimizer.step()

print("(CPU) training time :", time() - t_)

"""Price generation"""
t_ = time()
price = BS_price(S0, K, sig, r, T).float()
print("(CPU) price time :", time()-t_)


In [ ]:
S = torch.arange(50., 150., 0.2).view(-1,1).float()
S_numpy = S.numpy()
plt.figure()
plt.plot(S_numpy, BS_price(S_numpy, K, sig, r, T), color='r', alpha=0.5, label = "True price")
plt.plot(S_numpy, NET(S).detach().numpy(), color='b', alpha = 0.5, label = "NET price")
plt.legend()
plt.xlabel("S0")
plt.ylabel("price")
plt.grid()

In [ ]:
                                       """ GPU version """
K = torch.tensor(100, device = device)
#K = 100
T = torch.tensor(1, device = device)
#T = 1
sig = torch.tensor(0.2, device = device)
#sig = 0.2
r = torch.tensor(0.1, device = device)
#r = 0.1

#NET = net(1, 200, 200, 200, 1).cuda(device)
NET = net(1, 100, 100, 1).cuda(device)
optimizer = Adam(NET.parameters(), lr = 0.0005) 
loss_func = nn.MSELoss(reduction = 'mean')

#data
t_ = time()
S0 = torch.tensor(np.random.uniform(S0_min, S0_max, batch_size*1024*2), dtype=torch.float, device = device)
torch.cuda.synchronize()
print("(GPU) S0 generation time :", time()-t_)


S0 = S0.view(-1,1)
dataloader = torch.utils.data.DataLoader(S0, batch_size=batch_size, shuffle=True)

t_ = time()
for data in dataloader :
    optimizer.zero_grad()
    TRUE_price = price = BS_price_GPU(data, K, sig, r, T).float()
    NET_price = NET(data)
    
    loss = loss_func(TRUE_price, NET_price)
    loss.backward()
    optimizer.step()
    
torch.cuda.synchronize()
print("(GPU) training time :", time() - t_)

t_ = time()
price = BS_price_GPU(S0, K, sig, r, T).float()
print("(GPU) price time :", time()-t_)


In [ ]:

S = torch.arange(50., 150., 0.2, device = device).view(-1,1).float()
plt.figure()
plt.plot(S_numpy, BS_price_GPU(S, K, sig, r, T).cpu().numpy(), color='r', alpha=0.5, label = "True price")
plt.plot(S_numpy, NET(S).detach().cpu().numpy(), color='b', alpha = 0.5, label = "NET price")
plt.legend()
plt.xlabel("S0")
plt.ylabel("price")
plt.grid()